In [11]:

import sys

sys.path.insert(0, '/home/george/Projects/Python/bind-af2-embed/src')

# Import dependencies
%load_ext autoreload
%autoreload 2

import pandas as pd
from config import AppConfig
from plots import Plots

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Read predictions files

In [42]:
tag = '202207041511_CNN1D_EMBEDDINGS'

config = AppConfig()
predictions_path = config.get_ml_predictions_path()
predictions_path = predictions_path / tag / 'test_norm.csv'
df = pd.read_csv(str(predictions_path))

In [43]:
counts = df[['target', 'prediction']].value_counts()
tp = counts.loc[(1,1)] / sum(counts)
fp = counts.loc[(0,1)] / sum(counts)
tn = counts.loc[(0,0)] / sum(counts)
fn = counts.loc[(1,0)] / sum(counts)

In [44]:
df[df.target == 1]

,tag,protd_id,position,ligand,target,prediction,cutoff,ri,prob,loss
99,test,P02281,99,metal,1.0,0,0.5,7.488000,0.084,2729.514
100,test,P02281,100,metal,1.0,0,0.5,8.082000,0.051,3387.816
102,test,P02281,102,metal,1.0,0,0.5,8.190000,0.045,3552.204
105,test,P02281,105,metal,1.0,0,0.5,2.214000,0.377,1602.228
108,test,P02281,108,metal,1.0,0,0.5,8.784000,0.012,4361.887
...,...,...,...,...,...,...,...,...,...,...
250647,test,P40347,52,binding,1.0,1,0.5,8.028000,0.946,10.029
250692,test,P40347,97,binding,1.0,1,0.5,7.308001,0.906,18.225
250695,test,P40347,100,binding,1.0,0,0.5,6.858000,0.119,415.518
250727,test,P40347,132,binding,1.0,1,0.5,0.954000,0.553,115.693


In [45]:
df.loss.mean()

136.207361801911

In [46]:
 df.ligand.unique()

array(['metal', 'nuclear', 'small', 'binding'], dtype=object)

In [47]:
df

,tag,protd_id,position,ligand,target,prediction,cutoff,ri,prob,loss
0,test,P02281,0,metal,0.0,0,0.5,6.138001,0.159,22.311
1,test,P02281,1,metal,0.0,0,0.5,6.066000,0.163,23.056
2,test,P02281,2,metal,0.0,0,0.5,6.318000,0.149,20.776
3,test,P02281,3,metal,0.0,0,0.5,8.604000,0.022,2.780
4,test,P02281,4,metal,0.0,0,0.5,8.676000,0.018,2.329
...,...,...,...,...,...,...,...,...,...,...
250751,test,P40347,156,binding,0.0,0,0.5,9.000000,0.000,0.063
250752,test,P40347,157,binding,0.0,0,0.5,9.000000,0.000,0.022
250753,test,P40347,158,binding,0.0,0,0.5,9.000000,0.000,0.063
250754,test,P40347,159,binding,0.0,0,0.5,8.982000,0.001,0.170
